In [189]:
import pandas as pd

In [190]:
# Read data using pandas
dfpop = pd.read_csv     (r'Data/populationdata.csv')
dfpop.head()

,City,State,population,Lon,Lat
0,Henderson,Nevada,260068,-115.0375,36.0122
1,Manchester,New Hampshire,109830,-71.4439,42.9847
2,Elizabeth,New Jersey,125660,-74.1935,40.6663
3,Newark,New Jersey,277540,-74.1726,40.7242
4,Paterson,New Jersey,146427,-74.1628,40.9147


In [191]:
dftemp = pd.read_csv (r'Data/temperaturedata.csv')
dftemp.head()

,name,country_name,country_code,continent,station_code,location_date,temp_mean_c,temp_min_c,temp_max_c
0,Atlanta,UnitedStates,USA,North America,KATL,4/20/2021,17.491667,12.2,23.3
1,Windsor Locks,UnitedStates,USA,North America,KBDL,4/20/2021,13.887500,3.9,22.2
2,Nashville,UnitedStates,USA,North America,KBNA,4/20/2021,15.175000,6.7,23.3
3,Boise,UnitedStates,USA,North America,KBOI,4/20/2021,9.329167,1.7,15.6
4,Boston,UnitedStates,USA,North America,KBOS,4/20/2021,16.716667,10.0,23.9


In [192]:
#check for NaN in dftemp
NaNsum = dftemp.isna().sum().sum()

In [193]:
#if NaN > 0, drop
if(NaNsum > 0): 
    dftemp.dropna()
    df.head()
else: 
    print('No NaNs')

No NaNs


In [194]:
from sklearn import neighbors 
from sklearn.model_selection import train_test_split

#drop name, country_name, country_code, continent
dftemp.drop(columns=[
    'name', 
    'country_name', 
    'country_code', 
    'continent'
], inplace=True)
dftemp.head()

,station_code,location_date,temp_mean_c,temp_min_c,temp_max_c
0,KATL,4/20/2021,17.491667,12.2,23.3
1,KBDL,4/20/2021,13.887500,3.9,22.2
2,KBNA,4/20/2021,15.175000,6.7,23.3
3,KBOI,4/20/2021,9.329167,1.7,15.6
4,KBOS,4/20/2021,16.716667,10.0,23.9


In [195]:
#need to replace dates with datetime objects
import seaborn as sns 
import datetime as dt


dftemp['date_parsed'] = pd.to_datetime(dftemp['location_date'], infer_datetime_format=True)

#drop old dates
dftemp.drop(columns=['location_date'], inplace=True)



In [196]:
#group by station
grouped = dftemp.groupby(dftemp.station_code)
df_KATL = grouped.get_group("KATL")
df_KATL

,station_code,temp_mean_c,temp_min_c,temp_max_c,date_parsed
0,KATL,17.491667,12.2,23.3,2021-04-20
75,KATL,16.154167,11.1,21.7,2021-04-19
78,KATL,16.720833,11.7,22.2,2021-04-18
153,KATL,16.795833,12.8,21.7,2021-04-17
156,KATL,14.695833,10.0,19.4,2021-04-16
...,...,...,...,...,...
89025,KATL,5.045833,1.1,8.9,2015-01-05
89100,KATL,14.279167,6.7,17.8,2015-01-04
89103,KATL,11.154167,8.9,16.1,2015-01-03
89178,KATL,8.116667,6.7,9.4,2015-01-02


In [197]:
y = df_KATL.temp_mean_c.values
df_KATL.drop(columns=['temp_mean_c', 'date_parsed', 'station_code'], inplace=True)
x = df_KATL.values

/Users/cadenwestmoreland/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [198]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state=1)

xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((1829, 2), (458, 2), (1829,), (458,))

In [199]:
#make linear regression
from sklearn import linear_model

reg = linear_model.LinearRegression()
reg.fit(xtrain, ytrain)
reg.coef_

array([0.47006815, 0.51854193])

In [200]:
#check score on train and test sets. best possible = 1.0
reg.score(xtrain, ytrain), reg.score(xtest, ytest)


(0.9938965432680634, 0.9944244965363676)

In [ ]:
#we can repeat this process for every group in grouped

